# CS205 Project 2
### Divyank Shah (dshah048)
### 14th June 2024

Github Link: https://github.com/shahdivyank/cs205_project_2

In order to implement this project, I referenced the following sites/resouces:

1. Numpy Documentation: https://numpy.org/doc/stable/index.html 
2. Pandas Documentation: https://pandas.pydata.org/docs/
3. SkLearn Documentation: https://scikit-learn.org/stable/ 
4. Slides (Project 2 Briefing) + Example report provided

I affirm that I did not use ChatGPT or similar to write the code or text in this work.

## Introduction 

The following projects attempts to depicts the forward and backward feature selection algorithms on varying sized datasets with numerous features. The small dataset consists of 12 features and 500 datapoints while the large dataset consists of 50 features and 5000 datapoints. 

The following report dives into the project assigned in CS 205: Artificial Intelligence by Professor Keogh in Spring 2024 at the University of California, Riverside. The write up includes information about the implementation, various bugs, and optimizations.

In [5]:
# importing necessary libraries

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

/var/folders/hq/zydtttq12xq05qpxw71t17200000gn/T/ipykernel_44614/1190984141.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
# I was assigned Small_40 and Large_15 as per the project description requirements

SMALL_FILENAME = "./data/CS205_small_Data__40.txt"
LARGE_FILENAME = "./data/CS205_large_Data__15.txt"

In [7]:
# parse both data files into dataframes using the read_fwf function which takes in a fixed width based data file
# https://stackoverflow.com/questions/68871370/parse-problematic-fixed-width-text-file-to-a-pandas-dataframe
# https://pandas.pydata.org/pandas-docs/version/1.2.0/reference/api/pandas.read_fwf.html

small_df = pd.read_fwf(SMALL_FILENAME, header = None)
large_df = pd.read_fwf(LARGE_FILENAME, header = None)

In [8]:
# Separate the X (features) and Y (labels) for each small and large dataset and convert to numpy for faster execution

X_SMALL = small_df.iloc[:, 1:].to_numpy()
Y_SMALL = small_df.iloc[:, 0].to_numpy()

X_LARGE = large_df.iloc[:, 1:].to_numpy()
Y_LARGE = large_df.iloc[:, 0].to_numpy()

In [9]:
# Helper function to take the distance matrix provided and find the smallest distance. Ensures that it does not return itself, 
# by setting its own distance to infinity and reverting it back afterwards 

def nearest_neighbor(distances, index):
    distances[index][index] = np.inf
    shortest_distance_index = np.argmin(distances[index])
    distances[index][index] = 0

    return shortest_distance_index

In [10]:
# Helper function to find the distance between all datapoints with selected features.
# Based on the pairwise distance matrix, it will get the nearest neighbor and the true label. Aggregating this information
# is then used to calculate the accuracy for the given set of features.

def calculate_accuracy(X, Y, features):
    correct_classified = 0

    distances = pairwise_distances(X[:, features])

    for datapoint_index in range(X.shape[0]):
        nearest_neighbor_index = nearest_neighbor(distances, datapoint_index)
        classified_class = Y[nearest_neighbor_index]
        correct_class = Y[datapoint_index]

        if classified_class == correct_class:
            correct_classified += 1

    return correct_classified / X.shape[0]

## Forward Feature Selection

The forward selection algorithm is a method of starting from no selected features and adding a single feature each iteration. The feature with the highest accuracy is then chosen and inputed into a local maxima features list. This process repeats until all features have been selected. During this process, the global maxima is also kept track of and ultimately reported at the end. 

In [11]:
# The primary forward feature selection algortithm. Originally starting with an empty feature list and
# adding features iteratively and choosing the path with the highest accuracy and expanding from there.
# Global and local maximums are tracked to ensure the most optimal solution is found after finishing the
# algorithms.

def forward_feature_selection(X, Y):
    current_features = []
    global_accuracy = -np.inf
    global_features = []
    
    for i in range(X.shape[1]):
        print(f"On the {i}th level of the search tree")
        feature_to_add = None
        best_accuracy = 0.0

        for k in range(X.shape[1]):
            if k not in current_features:
                accuracy = calculate_accuracy(X, Y, current_features + [k])
                print(f'- Adding the {k + 1} feature with accuracy: {accuracy}')
            
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    feature_to_add = k
            
        if best_accuracy >= global_accuracy:
            global_accuracy = best_accuracy
            global_features.append(feature_to_add)
        else:
            print("\n**Warning, Accuracy has decreased! Continuing search in case of local maxima**")

        current_features.append(feature_to_add)
        print(f'\nOn level {i}, added feature {feature_to_add} to current set. With current features: {current_features} and local maximum accuracy {best_accuracy}\n')
    print(f"\n\nFinished. The best subset is {global_features} (indexes) {[x + 1 for x in global_features]} (features), which has an accuracy of {global_accuracy}")

## Backward Feature Selection

The backward feature algorithm operates by initially selecting all features and then removing exactly one feature. The feature set with n - 1 features with the highest accuracy is reduced further by another feature until there exists only a single feature. During the various iterations the local and global maxima are tracked and reported at the end. 

In [12]:
# The primary backward feature selection algortithm. Originally starting with an full feature list and
# removing features iteratively and choosing the path with the highest accuracy and expanding from there.
# Global and local maximums are tracked to ensure the most optimal solution is found after finishing the
# algorithms.

def backward_feature_selection(X, Y):
    current_features = [i for i in range(X.shape[1])]
    global_features = [i for i in range(X.shape[1])]

    global_accuracy = calculate_accuracy(X, Y, current_features)
    
    for i in range(X.shape[1]):
        print(f"On the {i}th level of the search tree")
        feature_to_remove = None
        best_accuracy = 0.0

        for k in range(X.shape[1]):
            if k in current_features and len(current_features) > 1:
                accuracy = calculate_accuracy(X, Y, list(set(current_features) - set([k])))

                print(f'- Removing the {k} feature with accuracy: {accuracy}')

                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    feature_to_remove = k
            
        if best_accuracy >= global_accuracy and feature_to_remove in global_features:
            global_accuracy = best_accuracy
            global_features.remove(feature_to_remove)
        else:
            print("\n**Warning, Accuracy has decreased! Continuing search in case of local maxima**")

        if feature_to_remove is not None:
            current_features.remove(feature_to_remove)
        print(f'\nOn level {i}, removed feature {feature_to_remove} to current set. With current features: {current_features} and local maximum accuracy {best_accuracy}\n')
    print(f"\n\nFinished. The best subset is {global_features} (indexes) {[x + 1 for x in global_features]} (features), which has an accuracy of {global_accuracy}")

In [21]:
%%capture cap
# Testbench for small dataset on SMALL_40 with forward feature selection.

forward_feature_selection(X_SMALL, Y_SMALL)

In [20]:
%%capture cap
# Testbench for small dataset on SMALL_40 with backward feature selection.

backward_feature_selection(X_SMALL, Y_SMALL)

In [19]:
%%capture cap
# Testbench for small dataset on LARGE_15 with forward feature selection.

forward_feature_selection(X_LARGE, Y_LARGE)

In [18]:
%%capture cap

# Testbench for small dataset on LARGE_15 with backward feature selection.

backward_feature_selection(X_LARGE, Y_LARGE)

# Conclusion

Using the traces provided in the `/results` folder, the accuracies across various iterations, the accuracy of removing or adding a feature can be seen and analyzed further if desired. 

### Small Dataset 40
Results for the small dataset were consistent were at least 2 of the expected 3 features were selected. Due to the initial separation of features and labels into X and Y, the features shown are the feature indexes which is off by one relative to the true feature number. Results for forward and backward feature selection are consistent however they appear in a different order. 

### Large Dataset 15
Results for the large dataset were consistent were at least 2 of the expected 3 features were selected in forward feature selection. Due to the initial separation of features and labels into X and Y, the features shown are the feature indexes which is off by one relative to the true feature number. Backwards feature selection does appear to have a bug which is only apparent when testing with larger datasets. It outputs to have multiple variables chosen when in reality it should have had only 2-3 variables. However, the expected features are included, accounting for the off by 1 index due to the X and Y split. 
